<a href="https://colab.research.google.com/github/151ali/lr-pytorch/blob/main/11_conv_autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torch.optim as optim
from torchvision import transforms

from torch.utils.data import DataLoader
from torchvision.utils import save_image, make_grid

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"running on {device}")

running on cuda


In [3]:
class AutoEncoder(nn.Module):
  def __init__(self):
    super(AutoEncoder, self).__init__()

    self.encoder = nn.Sequential(
        nn.Conv2d(1, 16, kernel_size=3, padding=1, bias=False),
        nn.BatchNorm2d(16),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(2, stride=2),
        nn.Conv2d(16, 8, kernel_size=3, padding=1, bias=False),
        nn.BatchNorm2d(8),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(2, stride=2),
        
    )

    self.decoder = nn.Sequential(
        nn.ConvTranspose2d(8,16, kernel_size=4, padding=1, stride=2),
        nn.ReLU(inplace=True),
        nn.ConvTranspose2d(16,1, kernel_size=3, padding=1, stride=2, output_padding=1),
        nn.ReLU(inplace=True),
        
    )


  def forward(self, x):
    e = self.encoder(x)
    d = self.decoder(e)
    return d

In [4]:
num_epochs = 100
batch_size = 64
learning_rate = 3e-4

In [5]:
tr = transforms.Compose([
    transforms.ToTensor(),
])

In [7]:
train_dataset = datasets.MNIST(root="./data", train=True, transform=tr, download=True);
test_dataset = datasets.MNIST(root="./data", train=False, transform=tr, download=True);
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [12]:
model = AutoEncoder().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(
    model.parameters(),
    lr = learning_rate,
)

In [ ]:
import tqdm

for epoch in range(num_epochs):
  for _ , (data, _) in enumerate(tqdm.tqdm(train_loader)):
    data = data.to(device)
    targets = targets.to(device)

    output = model(data)
    loss = criterion(output, data)
    optimizer.zero_grad()
    
    loss.backward()
    optimizer.step()

torch.save(model.state_dict(), '/content/drive/MyDrive/deep-learning/conv_mnist_autoencoder.pth')